In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()


In [ ]:
#Creating the ds_jobs_transformed

#Modifying the 'experience' column
ds_jobs['experience'] = ds_jobs['experience'].astype('category')

categories_exp = [
    '<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
    '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'
]

ds_jobs['experience'] = ds_jobs['experience'].cat.reorder_categories(new_categories=categories_exp, ordered=True)

ds_jobs['experience_codes'] = ds_jobs['experience'].cat.codes
ds_jobs['experience_codes'] = ds_jobs['experience_codes'].astype(int)

#Create the condition1 for subseting
condition1 = ds_jobs['experience_codes'] >= 10


#Modifying the "company_size" column
ds_jobs['company_size'] = ds_jobs['company_size'].astype('category')

categories_cmpn_size  = [
    '<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999','10000+'
]

ds_jobs['company_size'] = ds_jobs['company_size'].cat.reorder_categories(new_categories=categories_cmpn_size, ordered=True)

ds_jobs['company_size_codes'] = ds_jobs['company_size'].cat.codes
ds_jobs['company_size_codes'] = ds_jobs['company_size_codes'].astype(int)

#Create the condition2 for subseting
condition2 = ds_jobs['company_size_codes'] >= 5

# Combine conditions using np.logical_and()
combined_condition = np.logical_and(condition1, condition2)

#Create the new dataset
ds_jobs_transformed = ds_jobs[combined_condition]

ds_jobs_transformed.drop(columns=['experience_codes', 'company_size_codes'], inplace=True)

print(ds_jobs_transformed.head())


In [ ]:
#Modifying the 'last_new_job' column
ds_jobs_transformed['last_new_job'].value_counts(dropna=False)

categories = [
    'never', '1', '2', '3', '4', '>4'
]
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].astype('category')
ds_jobs_transformed['last_new_job'] = ds_jobs_transformed['last_new_job'].cat.reorder_categories(new_categories=categories, ordered=True)


In [ ]:
#Modifying 'relevant_experience' column bool dtype
ds_jobs_transformed['relevant_experience'] = np.where(ds_jobs_transformed['relevant_experience'].str.contains("Has", regex=False), 1, 0)
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype(bool)


In [ ]:
#Modifying 'job_change' column bool dtype
ds_jobs_transformed.loc[ds_jobs_transformed['job_change']==1.0, 'job_change'] = 1
ds_jobs_transformed.loc[ds_jobs_transformed['job_change']==0.0, 'job_change'] = 0
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype(bool)



In [ ]:
#Modifying for nominal categorical dtype cols
#Modifying 'major_discipline' column
ds_jobs_transformed['major_discipline'] = ds_jobs_transformed['major_discipline'].astype('category')

#Modifying 'company_type' column
ds_jobs_transformed['company_type'] = ds_jobs_transformed['company_type'].astype('category')

#Modifying 'education_level' column
categories_education_level = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].astype('category')
ds_jobs_transformed['education_level'] = ds_jobs_transformed['education_level'].cat.reorder_categories(new_categories=categories_education_level, ordered=True)

#Modifying 'gender' column
ds_jobs_transformed['gender'] = ds_jobs_transformed['gender'].astype('category')

#Modifying 'city' column
ds_jobs_transformed['city'] = ds_jobs_transformed['city'].astype('category')

#Modifying 'enrolled_university' column
categories_enrolled_uni = ['no_enrollment', 'Part time course', 'Full time course']
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].astype('category')
ds_jobs_transformed['enrolled_university'] = ds_jobs_transformed['enrolled_university'].cat.reorder_categories(new_categories=categories_enrolled_uni, ordered=True)


In [ ]:
#Modifying the 'student_id' column to int32
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('Int32')

#Modifying the 'training_hours' column to int32
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('Int32')

#Modifying the 'city_developement_index' column to float16
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

In [ ]:
#Print to see the difference in memory usage
print(ds_jobs.info)
print(ds_jobs_transformed.info)

In [ ]:
###~~~SOLUTION~~~###

# Import necessary libraries
import pandas as pd

# Load the dataset and create a copy
ds_jobs = pd.read_csv("customer_train.csv")
ds_jobs_transformed = ds_jobs.copy()

# EDA to help identify ordinal, nominal, and two-factor categories
for col in ds_jobs.select_dtypes("object").columns:
    print(ds_jobs_transformed[col].value_counts(), '\n')

# Create a dictionary of columns containing ordered categorical data
ordered_cats = {
    'enrolled_university': ['no_enrollment', 'Part time course', 'Full time course'],
    'education_level': ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'],
    'experience': ['<1'] + list(map(str, range(1, 21))) + ['>20'],
    'company_size': ['<10', '10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+'],
    'last_new_job': ['never', '1', '2', '3', '4', '>4']
}

# Create a mapping dictionary of columns containing two-factor categories to convert to Booleans
two_factor_cats = {
    'relevant_experience': {'No relevant experience': False, 'Has relevant experience': True},
    'job_change': {0.0: False, 1.0: True}
}

# Loop through DataFrame columns to efficiently change data types
for col in ds_jobs_transformed:
    
    # Convert two-factor categories to bool
    if col in ['relevant_experience', 'job_change']:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].map(two_factor_cats[col])
    
    # Convert integer columns to int32
    elif col in ['student_id', 'training_hours']:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('int32')
    
    # Convert float columns to float16
    elif col == 'city_development_index':
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('float16')
    
    # Convert columns containing ordered categorical data to ordered categories using dict
    elif col in ordered_cats.keys():
        category = pd.CategoricalDtype(ordered_cats[col], ordered=True)
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype(category)
    
    # Convert remaining columns to standard categories
    else:
        ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')
        
# Filter students with 10 or more years experience at companies with at least 1000 employees
ds_jobs_transformed = ds_jobs_transformed[(ds_jobs_transformed['experience'] >= '10') & (ds_jobs_transformed['company_size'] >= '1000-4999')]